In [1]:
import backtrader as bt
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [9]:
# class VWAP(bt.Indicator):
#     lines = ('vwap',)
#     plotinfo = dict(subplot=False)
    
#     def next(self):
#         # Calculate cumulative sum of price*volume and volume up to the current bar
#         total_pv = sum(self.data.close[i] * self.data.volume[i] for i in range(len(self.data)))
#         total_vol = sum(self.data.volume[i] for i in range(len(self.data)))
#         self.lines.vwap[0] = total_pv / total_vol if total_vol != 0 else self.data.close[0]


class VWAPRollingIndicator(bt.Indicator):
    """
    Volume Weighted Average Price (VWAP) indicator, rolling calculation.
    """

    lines = ("vwap_rolling",)
    params = {"period": 14}
    plotinfo = {"subplot": False}
    plotlines = {"vwap_rolling": {"color": "green"}}

    def __init__(self) -> None:
        self.hlc = (self.data.high + self.data.low + self.data.close) / 3.0
        self.hlc_volume_sum = bt.ind.SumN(self.hlc * self.data.volume, period=self.p.period)
        self.volume_sum = bt.ind.SumN(self.data.volume, period=self.p.period)

        self.lines.vwap_rolling = bt.DivByZero(self.hlc_volume_sum, self.volume_sum, None) 


        
class VWAPStrategy(bt.Strategy):
    params = (
        ("stop_loss", 0.02),  # 2% stop loss
        ("take_profit", 0.04),  # 4% take profit
        ("risk", 0.01),  # 1% risk
        ("pivot_window", 5),  # 5 bars for pivot high/low
        ("rrr", 2),  # Risk-Reward Ratio
    )
    def __init__(self):
        # initialize the VWAP indicator on data feed
        self.vwap = VWAPRollingIndicator(self.data)
        self.crossover = bt.indicators.CrossOver(self.data.close, self.vwap)
        self.order = None
        self.prev_pivot_high = self.data.low[0]
        self.prev_pivot_low = self.data.high[0]
        
    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
        self.order = None
    
    def log(self, txt):
        dt = self.data.datetime.date(0)
        print(f'{dt.isoformat()} {txt}')
    
    def _calculate_size(self, price, sl_price):
        # Calculate the size of the position based on the risk percentage and stop loss
        size = (self.broker.getvalue() * self.params.risk) / abs(price - sl_price)
        return size
    
    def _update_pivots(self):
        if np.max(self.data.high.get(size=self.params.pivot_window * 2 + 1)) == self.data.high[self.params.pivot_window]:
            self.prev_pivot_high = self.data.high[self.params.pivot_window]   
        if np.min(self.data.low.get(size=self.params.pivot_window * 2 + 1)) == self.data.low[self.params.pivot_window]:
            self.prev_pivot_low = self.data.low[self.params.pivot_window]      

    def next(self):
        
        # if not in a market position, check for crossover from below
        if not self.position:
            if self.crossover > 0 and self.data.close[1] > self.data.open[1]:
                price = self.data.close[0]
                # sl_price = self.data.low[1]
                sl_price = price * (1 - self.params.stop_loss)
                tp_price = price + np.abs(sl_price - price) * self.params.rrr
                if tp_price < price:
                    self.log(f'Take profit price {tp_price:.2f} is less than entry price {price:.2f}, not placing order.')
                    return
                if sl_price > price:
                    self.log(f'Stop loss price {sl_price:.2f} is greater than entry price {price:.2f}, not placing order.')
                    return

                size = self._calculate_size(price, sl_price)
                self.order = self.buy_bracket(
                    exectype=bt.Order.Market, 
                    size=size,
                    price=price,
                    stopprice=sl_price,
                    limitprice=tp_price,
                    stopargs=dict(exectype=bt.Order.Stop),
                    limitargs=dict(exectype=bt.Order.Limit)
                )
                self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")
 
            # if in a position, check for crossover from above for exit
            elif self.crossover < 0 and self.data.close[1] < self.data.open[1]:
                price = self.data.close[0]
                # sl_price = self.data.high[1]
                sl_price = price * (1 + self.params.stop_loss)
                tp_price = price - np.abs(sl_price - price) * self.params.rrr
                if tp_price > price:
                    self.log(f'Take profit price {tp_price:.2f} is greater than entry price {price:.2f}, not placing order.')
                    return
                if sl_price < price:
                    self.log(f'Stop loss price {sl_price:.2f} is less than entry price {price:.2f}, not placing order.')
                    return
                size = self._calculate_size(price, sl_price)
                self.order = self.sell_bracket(
                    exectype=bt.Order.Market, 
                    size=size,
                    price=price,
                    stopprice=sl_price,
                    limitprice=tp_price,
                    stopargs=dict(exectype=bt.Order.Stop),
                    limitargs=dict(exectype=bt.Order.Limit)
                )
                self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")
        else:
            pass
            


In [61]:
# Download SPX500 data from Yahoo Finance
yf_params = {'start': '2018-01-01', 'end': '2024-01-01', 'interval': '1d', 'multi_level_index': False}
# data = yf.download(tickers="^GSPC", **yf_params)
# data = yf.download(tickers="DJIA", **yf_params)
# data = yf.download(tickers="^VIX", **yf_params)
# data = yf.download(tickers="^RUT", **yf_params)
# data = yf.download(tickers="^IXIC", **yf_params)
# data = yf.download(tickers="GOOG", **yf_params)
# data = yf.download(tickers="AAPL", **yf_params)
# data = yf.download(tickers="MSFT", **yf_params)
# data = yf.download(tickers="AMZN", **yf_params)
# data = yf.download(tickers="TSLA", **yf_params)
# data = yf.download(tickers="META", **yf_params)
data = yf.download(tickers="NFLX", **yf_params)
# data = yf.download(tickers="NVDA", **yf_params)
# data = yf.download(tickers="AMD", **yf_params)
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
# data.set_index('Date', inplace=True)

# data

[*********************100%***********************]  1 of 1 completed


In [62]:
cerebro = bt.Cerebro()
cerebro.addstrategy(VWAPStrategy, stop_loss=0.10, take_profit=0.04, risk=0.01, pivot_window=5, rrr=1.5)
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)
data_feed = bt.feeds.pandafeed.PandasData(
    dataname=data,
    datetime=0,
    openinterest=-1,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
)
cerebro.adddata(data_feed)

In [63]:
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.01)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
cerebro.addanalyzer(bt.analyzers.Returns)

results = cerebro.run()

2018-02-07 Bracket order placed: Entry 266.58, SL 239.92, TP 306.57
2018-02-08 BUY EXECUTED, 250.10
2018-02-09 SELL EXECUTED, 239.92
2018-02-09 Bracket order placed: Entry 253.85, SL 279.24, TP 215.77
2018-02-12 SELL EXECUTED, 257.95
2018-02-15 BUY EXECUTED, 280.27
2018-02-15 Bracket order placed: Entry 270.03, SL 243.03, TP 310.53
2018-02-16 BUY EXECUTED, 278.52
2018-03-05 SELL EXECUTED, 315.00
2018-03-20 Bracket order placed: Entry 313.26, SL 344.59, TP 266.27
2018-03-21 SELL EXECUTED, 316.48
2018-05-23 BUY EXECUTED, 344.72
2018-07-09 Bracket order placed: Entry 415.95, SL 374.36, TP 478.34
2018-07-10 BUY EXECUTED, 415.63
2018-07-17 SELL EXECUTED, 374.36
2018-08-22 Bracket order placed: Entry 338.49, SL 304.64, TP 389.26
2018-08-23 BUY EXECUTED, 339.17
2018-10-24 SELL EXECUTED, 301.83
2018-11-29 Bracket order placed: Entry 282.32, SL 254.09, TP 324.67
2018-11-30 BUY EXECUTED, 286.13
2018-12-20 SELL EXECUTED, 254.09
2019-01-24 Bracket order placed: Entry 320.60, SL 352.66, TP 272.51
2

In [64]:
strat = results[0]
sharpe_ratio = strat.analyzers.sharperatio.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()
trade_analysis = strat.analyzers.tradeanalyzer.get_analysis()
returns = strat.analyzers.returns.get_analysis()
print(f'Sharpe Ratio: {sharpe_ratio}')
print(f'Max Drawdown: {drawdown["max"]["drawdown"]}')
print(f'Number of Trades: {trade_analysis.total.closed}')
print(f'Winning Trades: {trade_analysis.won.total}')
print(f'Losing Trades: {trade_analysis.lost.total}')
print(f'Average Trade Return: {returns["rnorm"]}')

Sharpe Ratio: OrderedDict({'sharperatio': -0.26456809915814017})
Max Drawdown: 6.601871360095412
Number of Trades: 35
Winning Trades: 12
Losing Trades: 23
Average Trade Return: 0.002421682463170222
